# Build a Crew to Tailor Job Applications

Build a multi-agent system.  Adapted from CrewAI Lesson 7 of their Initial Course

You will need the following libraries:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

You might need the following libraries.  Some of the libraries might already be installed but the versions are incorrect.  The following section of pip installs are optional.  Hopefully, they'll work for you but if they don't please feel free to leave a comment and I will try to help.

In [ ]:
!pip install openai

In [ ]:
!pip install utils

In [ ]:
!pip install anaconda-cloud-auth --upgrade

In [ ]:
!pip install --force-reinstall -v "Pydantic==2.6.2"

In [ ]:
!pip install Cython

In [ ]:
!pip install --force-reinstall -v "blosc2==2.0.0"

I found it easier to install the following libraries one at a time to allow for debugging any versioning issues.

In [ ]:
!pip install crewai==0.28.8

In [ ]:
!pip install crewai_tools==0.1.6

In [ ]:
!pip install langchain_community==0.0.29

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import libraries, APIs and LLM

In [2]:
from crewai import Agent, Task, Crew

**Note**: 
- The video uses `gpt-4-turbo`, but due to certain constraints, and in order to offer this course for free to everyone, the code you'll run here will use `gpt-3.5-turbo`.
- You can use `gpt-4-turbo` when you run the notebook _locally_ (using `gpt-4-turbo` will not work on the platform)
- Thank you for your understanding!

In [3]:
import os
import openai

# save your keys to user/system environment varaibles on your local machine
def get_serper_api_key():
    """Retrieve the Serper API key from an environment variable."""
    api_key = os.getenv('SERPER_API_KEY')
    if not api_key:
        raise ValueError("OpenAI API key not found. Please set the OPENAI_API_KEY environment variable.")
    return api_key


def get_openai_api_key():
    """Retrieve the OpenAI API key from an environment variable."""
    api_key = os.getenv('OPENAI_API_KEY')
    if not api_key:
        raise ValueError("OpenAI API key not found. Please set the OPENAI_API_KEY environment variable.")
    return api_key

# Retrieve and set the API key
#openai.api_key = get_openai_api_key() 
openai_api_key = get_openai_api_key()

os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'
#os.environ["SERPER_API_KEY"] = get_serper_api_key()

## crewAI Tools

In [4]:
from crewai_tools import (
  FileReadTool,
  ScrapeWebsiteTool,
  MDXSearchTool,
  SerperDevTool
)

search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()
read_resume = FileReadTool(file_path='./your_resume.md')
semantic_search_resume = MDXSearchTool(mdx='./your_resume.md')

- Uncomment and run the cell below if you wish to view `fake_resume.md` in the notebook.

In [5]:
from IPython.display import display, Markdown
# from IPython.display import Markdown, display
display(Markdown("./your_resume.md"))

# Jon Carlisle

**Address:** Evanston, IL, 60202  
**Phone:** (847) 924-1339  
**E-mail:** ecocarlisle@gmail.com  
**LinkedIn:** [linkedin.com/in/joncarlisle](https://linkedin.com/in/joncarlisle)

## Summary

Engineering leader with a successful record of leading products from prototype to launch. I help businesses build and scale their software products. As an experienced technologist in startup environments, I know how to build highly efficient, collaborative teams that thrive at building cutting-edge technology that customers love.

## Work History

### Magnolia Capital, Chicago, IL
**VP of Engineering**  
*2022-06 – Present*

- Technical, hands-on engineering leader responsible for building, leading, and managing the engineering team for start-up real estate investment firm.
- Defined and drove the technical vision, strategy, and roadmap in alignment with the company's business goals.
- Built multi-family predictive analytics product that predicts rent growth more accurately than leading analytics platform, RealPage Market Analytics.

### Omnitracs/Solera, Chicago, IL
**Senior Director of Engineering**  
*2019-04 – 2022-05*

- Grew the engineering team from 10 to 30+ engineers through thoughtful recruiting and talent retention.
- Stabilized and modernized Omnitracs’ B2B SaaS load board and TMS system.
- Collaborated with executive leadership across the organization to curate 3 million unique locations and develop new data products.

### Trianz, CBIG Consulting, Chicago, IL
**Manager of Strategic Analytics**  
*2017-03 – 2019-04*

- Managed Data Science project for pharmaceutical company that reduced time to service data constituents by 80% with onboarding of Snowflake solution for managing data in the cloud.

### Rotary International, Evanston, IL
**Senior Project Manager**  
*2012-06 – 2016-06*

- Directed changes to project scope and cost, implementing appropriate change management processes to keep projects on track.
- Developed project plans and managed project scope using methodologies to guide projects from conceptualization to implementation.

## Education

**Northwestern University, Evanston, IL**  
*Master of Science: Predictive Analytics*  
*2014-01 – 2015-12*

**DePaul University, Chicago, IL**  
*Computer Science: Computer Science*  
*1998-01 – 1999-01*

**University of Iowa, Iowa City, IA**  
*Bachelor of Arts: Film and Broadcasting*  
*1990-09 – 1994-05*

## Skills & Abilities

- Highly collaborative
- Excellent communication skills
- Strong leadership skills in software development & deployment
- Proficient in Python and SQL
- Experience working in fast-pace, start-up environment
- Experience architecting managed solutions in Azure and AWS
- Agile mindset and experience implementing agile ceremonies
- Strong analytical, debugging & troubleshooting skills
- Experience mentoring less experienced engineers and staff
- Strong judgment, decision making & prioritizing skills


## Creating Agents

In [6]:
# Agent 1: Researcher
researcher = Agent(
    role="Tech Job Researcher",
    goal="Make sure to do amazing analysis on "
         "job posting to help job applicants",
    tools = [scrape_tool, search_tool],
    verbose=True,
    backstory=(
        "As a Job Researcher, your prowess in "
        "navigating and extracting critical "
        "information from job postings is unmatched."
        "Your skills help pinpoint the necessary "
        "qualifications and skills sought "
        "by employers, forming the foundation for "
        "effective application tailoring."
    )
)

In [7]:
# Agent 2: Profiler
profiler = Agent(
    role="Personal Profiler for Engineers",
    goal="Do increditble research on job applicants "
         "to help them stand out in the job market",
    tools = [scrape_tool, search_tool,
             read_resume, semantic_search_resume],
    verbose=True,
    backstory=(
        "Equipped with analytical prowess, you dissect "
        "and synthesize information "
        "from diverse sources to craft comprehensive "
        "personal and professional profiles, laying the "
        "groundwork for personalized resume enhancements."
    )
)

In [8]:
# Agent 3: Resume Strategist
resume_strategist = Agent(
    role="Resume Strategist for Engineers",
    goal="Find all the best ways to make a "
         "resume stand out in the job market.",
    tools = [scrape_tool, search_tool,
             read_resume, semantic_search_resume],
    verbose=True,
    backstory=(
        "With a strategic mind and an eye for detail, you "
        "excel at refining resumes to highlight the most "
        "relevant skills and experiences, ensuring they "
        "resonate perfectly with the job's requirements."
    )
)

In [9]:
# Agent 4: Interview Preparer
interview_preparer = Agent(
    role="Engineering Interview Preparer",
    goal="Create interview questions and talking points "
         "based on the resume and job requirements",
    tools = [scrape_tool, search_tool,
             read_resume, semantic_search_resume],
    verbose=True,
    backstory=(
        "Your role is crucial in anticipating the dynamics of "
        "interviews. With your ability to formulate key questions "
        "and talking points, you prepare candidates for success, "
        "ensuring they can confidently address all aspects of the "
        "job they are applying for."
    )
)

## Creating Tasks

In [10]:
# Task for Researcher Agent: Extract Job Requirements
research_task = Task(
    description=(
        "Analyze the job posting URL provided ({job_posting_url}) "
        "to extract key skills, experiences, and qualifications "
        "required. Use the tools to gather content and identify "
        "and categorize the requirements."
    ),
    expected_output=(
        "A structured list of job requirements, including necessary "
        "skills, qualifications, and experiences."
    ),
    agent=researcher,
    async_execution=True
)

In [11]:
# Task for Profiler Agent: Compile Comprehensive Profile
profile_task = Task(
    description=(
        "Compile a detailed personal and professional profile "
        "using the GitHub ({github_url}) URLs, and personal write-up "
        "({personal_writeup}). Utilize tools to extract and "
        "synthesize information from these sources."
    ),
    expected_output=(
        "A comprehensive profile document that includes skills, "
        "project experiences, contributions, interests, and "
        "communication style."
    ),
    agent=profiler,
    async_execution=True
)

- You can pass a list of tasks as `context` to a task.
- The task then takes into account the output of those tasks in its execution.
- The task will not run until it has the output(s) from those tasks.

In [12]:
# Task for Resume Strategist Agent: Align Resume with Job Requirements
resume_strategy_task = Task(
    description=(
        "Using the profile and job requirements obtained from "
        "previous tasks, tailor the resume to highlight the most "
        "relevant areas. Employ tools to adjust and enhance the "
        "resume content. Make sure this is the best resume even but "
        "don't make up any information. Update every section, "
        "inlcuding the initial summary, work experience, skills, "
        "and education. All to better reflrect the candidates "
        "abilities and how it matches the job posting."
    ),
    expected_output=(
        "An updated resume that effectively highlights the candidate's "
        "qualifications and experiences relevant to the job."
    ),
    output_file="tailored_resume.md",
    context=[research_task, profile_task],
    agent=resume_strategist
)

In [13]:
# Task for Interview Preparer Agent: Develop Interview Materials
interview_preparation_task = Task(
    description=(
        "Create a set of potential interview questions and talking "
        "points based on the tailored resume and job requirements. "
        "Utilize tools to generate relevant questions and discussion "
        "points. Make sure to use these question and talking points to "
        "help the candiadte highlight the main points of the resume "
        "and how it matches the job posting."
    ),
    expected_output=(
        "A document containing key questions and talking points "
        "that the candidate should prepare for the initial interview."
    ),
    output_file="interview_materials.md",
    context=[research_task, profile_task, resume_strategy_task],
    agent=interview_preparer
)


## Creating the Crew

In [14]:
job_application_crew = Crew(
    agents=[researcher,
            profiler,
            resume_strategist,
            interview_preparer],

    tasks=[research_task,
           profile_task,
           resume_strategy_task,
           interview_preparation_task],

    verbose=True
)

## Running the Crew

- Set the inputs for the execution of the crew.

In [15]:
job_application_inputs = {
    'job_posting_url': 'you can put URL here or you can save it to Google Drive and use that URL',
    'github_url': 'https://github.com/you',
    'personal_writeup': """writeup goes here"""
}

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [16]:
### this execution will take a few minutes to run
result = job_application_crew.kickoff(inputs=job_application_inputs)

 [DEBUG]: == Working Agent: Tech Job Researcher
 [INFO]: == Starting Task: Analyze the job posting URL provided (https://docs.google.com/document/d/1OjGBANPQd2xIYmPVczPBCO8lRCXtANxw9yC-Q89r-Bg/edit?usp=sharing) to extract key skills, experiences, and qualifications required. Use the tools to gather content and identify and categorize the requirements.
 [DEBUG]: == [Tech Job Researcher] Task output: 


 [DEBUG]: == Working Agent: Personal Profiler for Engineers
 [INFO]: == Starting Task: Compile a detailed personal and professional profile using the GitHub (https://github.com/ecocarlisle) URLs, and personal write-up (Jon is an accomplished Software
    Engineering Leader with 24 years of experience, specializing in
    managing remote and in-office teams, and expert in multiple
    programming languages and frameworks. He holds a MPA and a strong
    background in AI and data science. Jon has successfully led
    major tech initiatives and startups, proving his ability to drive
    inno

Thought: I need to gather more information about Jon from his GitHub repositories.

Action: Read website content
Action Input: {"website_url": "https://github.com/ecocarlisle?tab=repositories"} 

ecocarlisle (ecocarlisle) / Repositories · GitHub
Skip to content
Navigation Menu
Toggle navigation
 Sign in
 Product
Actions
 Automate any workflow
Packages
 Host and manage packages
Security
 Find and fix vulnerabilities
Codespaces
 Instant dev environments
Copilot
 Write better code with AI
Code review
 Manage code changes
Issues
 Plan and track work
Discussions
 Collaborate outside of code
Explore
 All features
 Documentation
 GitHub Skills
 Blog
 Solutions
For
 Enterprise
 Teams
 Startups
 Education
By Solution
 CI/CD & Automation
 DevOps
 DevSecOps
Resources
 Learning Pathways
 White papers, Ebooks, Webinars
 Customer Stories
 Partners
 Open Source
GitHub Sponsors
 Fund open source developers
The ReadME Project
 GitHub community articles
Repositories
 Topics
 Trending
 Collections
Pricin

Thought: I need to correct my mistake and use the available tools effectively to update the resume.

Action: Read a file's content
Action Input: {} 

# Jon Carlisle

**Address:** Evanston, IL, 60202  
**Phone:** (847) 924-1339  
**E-mail:** ecocarlisle@gmail.com  
**LinkedIn:** [linkedin.com/in/joncarlisle](https://linkedin.com/in/joncarlisle)

## Summary

Engineering leader with a successful record of leading products from prototype to launch. I help businesses build and scale their software products. As an experienced technologist in startup environments, I know how to build highly efficient, collaborative teams that thrive at building cutting-edge technology that customers love.

## Work History

### Magnolia Capital, Chicago, IL
**VP of Engineering**  
*2022-06 â€“ Present*

- Technical, hands-on engineering leader responsible for building, leading, and managing the engineering team for start-up real estate investment firm.
- Defined and drove the technical vision, strategy, and ro

I need to create a set of potential interview questions and talking points based on Jon Carlisle's resume and the job requirements for the Director of Software Engineering position at DLA Piper. I must tailor the questions to highlight Jon's relevant experience, skills, and achievements.

Action: Read the resume content

Action Input: {Read a file's content}
 

Action 'Read the resume content' don't exist, these are the only available Actions: Read website content: Read website content(website_url: 'string') - A tool that can be used to read a website content.
Search the internet: Search the internet(search_query: 'string') - A tool that can be used to semantic search a query from a txt's content.
Read a file's content: Read a file's content() - A tool that can be used to read ./jon_resume.md's content.
Search a MDX's content: A tool that can be used to semantic search a query the ./jon_resume.md MDX's content.
Delegate work to co-worker: Delegate work to co-worker(coworker: str, task:

Thought: I need to access Jon Carlisle's resume to create interview questions and talking points.

Action: Read the file's content

Action Input: {}
 

Action 'Read the file's content' don't exist, these are the only available Actions: Read website content: Read website content(website_url: 'string') - A tool that can be used to read a website content.
Search the internet: Search the internet(search_query: 'string') - A tool that can be used to semantic search a query from a txt's content.
Read a file's content: Read a file's content() - A tool that can be used to read ./jon_resume.md's content.
Search a MDX's content: A tool that can be used to semantic search a query the ./jon_resume.md MDX's content.
Delegate work to co-worker: Delegate work to co-worker(coworker: str, task: str, context: str) - Delegate a specific task to one of the following co-workers: [Tech Job Researcher, Personal Profiler for Engineers, Resume Strategist for Engineers]
The input to this tool should be the cowo

Final Answer:

Based on Jon Carlisle's resume and the job requirements for the Director of Software Engineering position at DLA Piper, here are some potential interview questions and talking points to help Jon highlight his relevant experience and skills:

1. Can you walk us through a specific project you led from prototype to launch, emphasizing how you managed the technical vision, strategy, and roadmap to align with business goals?
2. How have you successfully built and led collaborative teams in both remote and in-office settings, especially in fast-paced startup environments?
3. Describe a time when you implemented agile ceremonies and methodologies to drive continuous improvement initiatives in software development processes. What were the results?
4. Discuss your experience in architecting managed solutions in Azure and AWS, highlighting any innovative approaches or challenges you faced.
5. How have you mentored less experienced engineers and staff to enhance their skills and co

- Dislplay the generated `tailored_resume.md` file.

In [17]:
from IPython.display import Markdown, display
display(Markdown("./tailored_resume.md"))

# Jon Carlisle

**Address:** Evanston, IL, 60202  
**Phone:** (847) 924-1339  
**E-mail:** ecocarlisle@gmail.com  
**LinkedIn:** [linkedin.com/in/joncarlisle](https://linkedin.com/in/joncarlisle)

## Summary

Engineering leader with a successful record of leading products from prototype to launch. I help businesses build and scale their software products. As an experienced technologist in startup environments, I know how to build highly efficient, collaborative teams that thrive at building cutting-edge technology that customers love.

## Work History

### Magnolia Capital, Chicago, IL
**VP of Engineering**  
*2022-06 – Present*

- Technical, hands-on engineering leader responsible for building, leading, and managing the engineering team for start-up real estate investment firm.
- Defined and drove the technical vision, strategy, and roadmap in alignment with the company's business goals.
- Built multi-family predictive analytics product that predicts rent growth more accurately than leading analytics platform, RealPage Market Analytics.

### Omnitracs/Solera, Chicago, IL
**Senior Director of Engineering**  
*2019-04 – 2022-05*

- Grew the engineering team from 10 to 30+ engineers through thoughtful recruiting and talent retention.
- Stabilized and modernized Omnitracs’ B2B SaaS load board and TMS system.
- Collaborated with executive leadership across the organization to curate 3 million unique locations and develop new data products.

### Trianz, CBIG Consulting, Chicago, IL
**Manager of Strategic Analytics**  
*2017-03 – 2019-04*

- Managed Data Science project for pharmaceutical company that reduced time to service data constituents by 80% with onboarding of Snowflake solution for managing data in the cloud.

### Rotary International, Evanston, IL
**Senior Project Manager**  
*2012-06 – 2016-06*

- Directed changes to project scope and cost, implementing appropriate change management processes to keep projects on track.
- Developed project plans and managed project scope using methodologies to guide projects from conceptualization to implementation.

## Education

**Northwestern University, Evanston, IL**  
*Master of Science: Predictive Analytics*  
*2014-01 – 2015-12*

**DePaul University, Chicago, IL**  
*Computer Science: Computer Science*  
*1998-01 – 1999-01*

**University of Iowa, Iowa City, IA**  
*Bachelor of Arts: Film and Broadcasting*  
*1990-09 – 1994-05*

## Skills & Abilities

- Highly collaborative
- Excellent communication skills
- Strong leadership skills in software development & deployment
- Proficient in Python and SQL
- Experience working in fast-pace, start-up environment
- Experience architecting managed solutions in Azure and AWS
- Agile mindset and experience implementing agile ceremonies
- Strong analytical, debugging & troubleshooting skills
- Experience mentoring less experienced engineers and staff
- Strong judgment, decision-making & prioritizing skills

- Dislplay the generated `interview_materials.md` file.

In [ ]:
display(Markdown("./interview_materials.md"))

# CONGRATULATIONS!!!

## Share your accomplishment!
- Once you finish watching all the videos, you will see the "In progress" image on the bottom left turn into "Accomplished".
- Click on "Accomplished" to view the course completion page with your name on it.
- Take a screenshot and share on LinkedIn, X (Twitter), or Facebook.  
- **Tag @Joāo (Joe) Moura, @crewAI, and @DeepLearning.AI, (and a few of your friends if you'd like them to try out the course)**
- **Joāo and DeepLearning.AI will "like"/reshare/comment on your post!**

## Get a completion badge that you can add to your LinkedIn profile!
- Go to [learn.crewai.com](https://learn.crewai.com).
- Upload your screenshot of your course completion page.
- You'll get a badge from CrewAI that you can share!

(Joāo will also talk about this in the last video of the course.)